# Generating metrics for selected baselines

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import sys
env_path = "%s" % os.path.dirname(os.path.abspath(".")) 
sys.path.append(env_path)
import logging

import numpy as np
import pandas as pd
import scanpy as sc
from scipy import sparse
import scvi

from sc_foundation_evals import utils
from sc_foundation_evals.helpers.custom_logging import log
import anndata

log.setLevel(logging.INFO)

import warnings
os.environ["KMP_WARNINGS"] = "off"
warnings.filterwarnings("ignore")

Specifying necessary variables, including path to anndata and names of columns with cell type labels and batch labels. I will be using the Pancreas dataset as an example, as described in the scGPT_zer-shot notebook.

In [ ]:
# specify the path to anndata object
adata_path = "../data/datasets/pancreas_scib.h5ad"
# dataset_name is inferred from in_dataset_path
dataset_name = os.path.basename(adata_path).split(".")[0]

output_folder = "../output/HVG"

# batch column found in adata.obs
batch_col = "batch"
# where are labels stored in adata.obs? 
label_col = "celltype"
# where the raw counts are stored?
layer_key = "counts"

Reading the anndata.

In [29]:
adata = sc.read(adata_path)

If the raw data is stored in `X` or other layer instead of `counts`, we need to copy it to counts to be able to use it in scVI.

In [30]:
if layer_key == "X":
    adata.layers["counts"] = adata.X
elif layer_key != "counts":
    adata.layers["counts"] = adata.layers[layer_key]

Here, I opted for minimal preprocessing, similar to this suggested by Geneformer.

In [31]:
sc.pp.filter_cells(adata, min_genes=10)
print(adata.X.shape)
sc.pp.filter_genes(adata, min_cells=10)
print(adata.X.shape)
sc.pp.normalize_total(adata, target_sum=1e4)
# print(adata.X.sum(1))
sc.pp.log1p(adata)

(16382, 19093)
(16382, 17379)


## Highly variable genes

For first baseline, I selected 2000 highly variable genes (HVGs) using `scanpy.pp.highly_variable_genes` with default parameters. I do not want the adata to be subsetted, so I set `subset=False` and save the created cell embedding space to `adata.obsm['X_hvg']`.

In [32]:
sc.pp.highly_variable_genes(adata, flavor='seurat', subset=False, n_top_genes=2000)

adata.obsm["X_genes"] = adata.X[:, adata.var.highly_variable]

# check if adata.obsm["X_genes"] is sparse and if so, convert to dense
if sparse.issparse(adata.obsm["X_genes"]):
    adata.obsm["X_genes"] = np.asarray(adata.obsm["X_genes"].todense())

Calculating metrics similiarly to those calculated for Geneformer and scGPT.

In [ ]:
scib_metrics = utils.eval_scib_metrics(adata, 
                                       batch_key=batch_col, 
                                       label_key=label_col,
                                       embedding_key="X_genes")

In [ ]:
scib_metrics

In [ ]:
scib_metrics.to_csv(os.path.join(output_folder, "hpancreas_cluster.csv"))

To visualize, we will use the umap plotting function from scanpy.

In [ ]:
import matplotlib.pyplot as plt
sc.set_figure_params(facecolor="white", figsize=(5,4), transparent=True, frameon=False)
sc.pp.neighbors(adata, use_rep="X_genes")
sc.tl.umap(adata, min_dist = 0.3)
fig = sc.pl.umap(adata, color=[batch_col, label_col], wspace = 0.4, return_fig=True)
plt.tight_layout()
fig.savefig("../output/pancreas_scib/HVG/clustering.png", dpi=200, bbox_inches='tight')

### reference mapping (label transfer)

In [ ]:
import faiss
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix

In [ ]:
use_rep_key = "X_genes"
ref_batch = ['indrop', 'celseq2', 'celseq']
adata.obs["is_ref"] = adata.obs["batch"].isin(ref_batch)
adata.obs

In [ ]:
ref_adata = adata[adata.obs["is_ref"]==True]
test_adata = adata[adata.obs["is_ref"]==False]

In [ ]:
ref_cell_embeddings = ref_adata.obsm[use_rep_key]
test_embed = test_adata.obsm[use_rep_key]

k = 10  # number of neighbors

index = faiss.IndexFlatL2(ref_cell_embeddings.shape[1])
index.add(ref_cell_embeddings)

# Query dataset, k - number of closest elements (returns 2 numpy arrays)
distances, labels = index.search(test_embed, k)

idx_list=[i for i in range(test_embed.shape[0])]
preds = []
sim_list = distances
for k in idx_list:
    idx = labels[k]
    pred = ref_adata.obs[label_col][idx].value_counts()
    preds.append(pred.index[0])
gt = test_adata.obs[label_col].to_numpy()

In [ ]:
res_dict = {
    "accuracy": accuracy_score(gt, preds),
    "precision": precision_score(gt, preds, average="macro"),
    "recall": recall_score(gt, preds, average="macro"),
    "macro_f1": f1_score(gt, preds, average="macro"),
}

res_dict

## scVI

As the other baseline, we look at the scVI model, which is a VAE model. To read more about it please refer to [scvi-tools manual](https://docs.scvi-tools.org/en/stable/user_guide/models/scvi.html) or [its publication](https://www.nature.com/articles/s41592-018-0229-2/).

### Train the scVI model use full data

In [ ]:
SCVI_LATENT_KEY = "X_scVI"

In [ ]:
# select 2000 HVGs
sc.pp.highly_variable_genes(
    adata, n_top_genes=2000, batch_key=batch_col, subset=True
)

In [ ]:
adata.X.shape

In [ ]:
scvi.model.SCVI.setup_anndata(adata, layer="counts", batch_key=batch_col)

Specifying the model and training it.

In [ ]:
model = scvi.model.SCVI(adata, n_layers=2, n_latent=10, gene_likelihood="nb")

In [ ]:
model.train()

Saving the cell embedding space to `adata.obsm['X_scvi']`.

In [ ]:
adata.obsm[SCVI_LATENT_KEY] = model.get_latent_representation()

#### Clustering metrics

In [ ]:
scib_metrics = utils.eval_scib_metrics(adata, 
                                       batch_key=batch_col, 
                                       label_key=label_col,
                                       embedding_key=SCVI_LATENT_KEY)

In [ ]:
sc.pp.neighbors(adata, use_rep=SCVI_LATENT_KEY)
sc.tl.umap(adata, min_dist = 0.3)

#### Visualizing the cell embedding space.

In [ ]:
sc.pl.umap(adata, color=[batch_col, label_col], wspace = 0.4)

#### reference mapping

In [ ]:
import faiss
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix

In [ ]:
ref_batch = ['indrop', 'celseq2', 'celseq']
adata.obs["is_ref"] = adata.obs["batch"].isin(ref_batch)
adata.obs

In [ ]:
ref_adata = adata[adata.obs["is_ref"]==True]
test_adata = adata[adata.obs["is_ref"]==False]

In [ ]:
ref_cell_embeddings = ref_adata.obsm[SCVI_LATENT_KEY]
test_embed = test_adata.obsm[SCVI_LATENT_KEY]

k = 10  # number of neighbors

index = faiss.IndexFlatL2(ref_cell_embeddings.shape[1])
index.add(ref_cell_embeddings)

# Query dataset, k - number of closest elements (returns 2 numpy arrays)
distances, labels = index.search(test_embed, k)

idx_list=[i for i in range(test_embed.shape[0])]
preds = []
sim_list = distances
for k in idx_list:
    idx = labels[k]
    pred = ref_adata.obs[label_col][idx].value_counts()
    preds.append(pred.index[0])
gt = test_adata.obs[label_col].to_numpy()

In [ ]:
res_dict = {
    "accuracy": accuracy_score(gt, preds),
    "precision": precision_score(gt, preds, average="macro"),
    "recall": recall_score(gt, preds, average="macro"),
    "macro_f1": f1_score(gt, preds, average="macro"),
}

res_dict

### Train the scVI model use the reference data and update with query

In [ ]:
pancreas_ref = adata[adata.obs["is_ref"]==True]
pancreas_query = adata[adata.obs["is_ref"]==False]
assert pancreas_ref.X.shape[0]+pancreas_query.X.shape[0] == adata.X.shape[0]

In [ ]:
# select 2000 HVGs
sc.pp.highly_variable_genes(
    pancreas_ref, n_top_genes=2000, batch_key=batch_col, subset=True
)

pancreas_query = pancreas_query[:, pancreas_ref.var_names].copy()

In [ ]:
scvi.model.SCVI.setup_anndata(pancreas_ref, layer="counts", batch_key=batch_col)

In [ ]:
scvi_ref = scvi.model.SCVI(
    pancreas_ref,
    use_layer_norm="both",
    use_batch_norm="none",
    encode_covariates=True,
    dropout_rate=0.2,
    n_layers=2,
) 
scvi_ref.train()

In [ ]:
scvi.model.SCVI.prepare_query_anndata(pancreas_query, scvi_ref)

In [ ]:
scvi_query = scvi.model.SCVI.load_query_data(
    pancreas_query,
    scvi_ref,
) 

In [ ]:
#! weight_decay=0, make sure thet the latent representations for ref_data are fixed 
scvi_query.train(max_epochs=200, plan_kwargs={"weight_decay": 0.0}) 

In [ ]:
pancreas_full = anndata.concat([pancreas_query, pancreas_ref])
pancreas_full

In [ ]:
pancreas_full.obsm[SCVI_LATENT_KEY] = scvi_query.get_latent_representation(
    pancreas_full
)

In [ ]:
ref_cell_embeddings = pancreas_full.obsm[SCVI_LATENT_KEY][pancreas_full.obs["is_ref"]==True]
test_embed = pancreas_full.obsm[SCVI_LATENT_KEY][pancreas_full.obs["is_ref"]==False]

k = 10  # number of neighbors

index = faiss.IndexFlatL2(ref_cell_embeddings.shape[1])
index.add(ref_cell_embeddings)

# Query dataset, k - number of closest elements (returns 2 numpy arrays)
distances, labels = index.search(test_embed, k)

idx_list=[i for i in range(test_embed.shape[0])]
preds = []
sim_list = distances
for k in idx_list:
    idx = labels[k]
    pred = ref_adata.obs[label_col][idx].value_counts()
    preds.append(pred.index[0])
gt = test_adata.obs[label_col].to_numpy()

In [ ]:
res_dict = {
    "accuracy": accuracy_score(gt, preds),
    "precision": precision_score(gt, preds, average="macro"),
    "recall": recall_score(gt, preds, average="macro"),
    "macro_f1": f1_score(gt, preds, average="macro"),
}
res_dict

### Supervised using reference data labels (SCANVI)